# 1. Librarys

In [2]:
!python -m pip install -q imbalanced-learn==0.7.0

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


##### Create bucket

In [31]:
import os
import boto3
import re
import sagemaker

role = sagemaker.get_execution_role()
region = boto3.Session().region_name

# S3 bucket for saving code and model artifacts.
# Feel free to specify a different bucket and prefix
bucket = sagemaker.Session().default_bucket()

prefix = (
    "sagemaker/breast-cancer-prediction"  # place to upload training files within the bucket
)

In [32]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import io
import time
import json
from imblearn.over_sampling import SMOTE
import sagemaker.amazon.common as smac

# 2. Create bucket and download data

In [33]:
s3 = boto3.client("s3")

filename = "wdbc.csv"
s3.download_file("sagemaker-sample-files", "datasets/tabular/breast_cancer/wdbc.csv", filename)

data = pd.read_csv(filename, header=None)


In [34]:
data.columns = [
    "id",
    "diagnosis",
    "radius_mean",
    "texture_mean",
    "perimeter_mean",
    "area_mean",
    "smoothness_mean",
    "compactness_mean",
    "concavity_mean",
    "concave points_mean",
    "symmetry_mean",
    "fractal_dimension_mean",
    "radius_se",
    "texture_se",
    "perimeter_se",
    "area_se",
    "smoothness_se",
    "compactness_se",
    "concavity_se",
    "concave points_se",
    "symmetry_se",
    "fractal_dimension_se",
    "radius_worst",
    "texture_worst",
    "perimeter_worst",
    "area_worst",
    "smoothness_worst",
    "compactness_worst",
    "concavity_worst",
    "concave points_worst",
    "symmetry_worst",
    "fractal_dimension_worst",
]

# save the data
data.to_csv("data.csv", sep=",", index=False)
print(data.shape)
display(data.head())
display(data.describe())
display(data.diagnosis.value_counts())

(569, 32)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


,id,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
count,5.690000e+02,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,3.037183e+07,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,1.250206e+08,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,8.670000e+03,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,8.692180e+05,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,9.060240e+05,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,8.813129e+06,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,9.113205e+08,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


B    357
M    212
Name: diagnosis, dtype: int64

###### I convert  the column of diagnosis to numeric values, where M = 1 and B = 0, I make this because the function SMOTE use numeric values, the categoric values don't work with this function

In [35]:
data['diagnosis'] = data['diagnosis'].apply(lambda x: 1 if x == 'M' else 0)

In [36]:
data

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,926424,1,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,926682,1,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,926954,1,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,927241,1,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


# 3. Split data, normalize  and use SMOTE

In [38]:
#80%  train, 10% validation, 10% test
rand_split = np.random.rand(len(data))
train_list = rand_split < 0.8
val_list = (rand_split >= 0.8) & (rand_split < 0.9)
test_list = rand_split >= 0.9

In [39]:
data_train = data[train_list]
data_train = data_train.reset_index(drop = True)
data_val = data[val_list]
data_val = data_val.reset_index(drop = True)
data_test = data[test_list]
data_test = data_test.reset_index(drop = True)

diagnosis = data_train['diagnosis']
diagnosis.value_counts()


0    287
1    168
Name: diagnosis, dtype: int64

###### standardizes the data

In [41]:
ss = StandardScaler()
ss.fit(data_train)

StandardScaler()

In [42]:
columns_df = np.array(data.columns)
columns_df

array(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean',
       'concavity_mean', 'concave points_mean', 'symmetry_mean',
       'fractal_dimension_mean', 'radius_se', 'texture_se',
       'perimeter_se', 'area_se', 'smoothness_se', 'compactness_se',
       'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst'], dtype=object)

In [43]:
data_train_standarized = ss.transform(data_train)
data_val_standarized = ss.transform(data_val)
data_test_standarized = ss.transform(data_test)

In [44]:
#np array to data frame
data_train_df  = pd.DataFrame(data_train_standarized, columns = columns_df)
data_val_df  = pd.DataFrame(data_val_standarized, columns = columns_df)
data_test_df  = pd.DataFrame(data_test_standarized, columns = columns_df)
data_train_df['id'] = data_train['id']
data_train_df['diagnosis'] = data_train['diagnosis']
data_val_df['id'] = data_val['id']
data_val_df['diagnosis'] = data_val['diagnosis']
data_test_df['id'] = data_test['id']
data_test_df['diagnosis'] = data_test['diagnosis']


In [45]:
data_train_df

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,1.094489,-2.023251,1.265430,0.993147,1.521799,3.313869,2.691176,2.489123,...,1.887671,-1.320509,2.294064,2.016539,1.291367,2.625945,2.122287,2.247068,2.913269,2.004583
1,842517,1,1.830409,-0.347405,1.682332,1.929473,-0.826243,-0.496955,-0.017233,0.528264,...,1.806666,-0.352456,1.527499,1.904880,-0.382119,-0.427414,-0.132465,1.058600,-0.242728,0.302907
2,84300903,1,1.579398,0.441762,1.562628,1.575109,0.907932,1.059476,1.386516,1.999737,...,1.511726,-0.014911,1.340314,1.467107,0.515636,1.089067,0.868582,1.911782,1.228707,0.220898
3,84348301,1,-0.779540,0.244470,-0.601137,-0.778382,3.202958,3.434537,1.945474,1.421140,...,-0.286989,0.139532,-0.253131,-0.555688,3.365793,3.905296,2.002430,2.128820,6.386399,5.085597
4,84358402,1,1.750542,-1.125233,1.773143,1.845924,0.259186,0.540411,1.394138,1.398200,...,1.297791,-1.425594,1.331401,1.229611,0.210574,-0.310124,0.626950,0.706849,-0.900866,-0.394165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
450,925622,1,0.304373,2.566617,0.464647,0.174654,0.573095,1.994178,2.118243,1.143056,...,0.255118,2.733215,0.633173,0.059853,0.397969,3.429763,4.318548,1.801018,2.040297,3.257491
451,926125,1,1.930243,1.312567,2.099235,1.989974,0.928859,2.279568,2.910948,2.496770,...,1.661273,0.602860,2.130649,1.662067,0.354389,1.051457,1.872984,2.079425,0.062471,0.855895
452,926424,1,2.112797,0.700282,2.057957,2.370266,1.005592,0.216713,1.977233,2.280115,...,1.902210,0.123610,1.744395,2.030718,0.367463,-0.269965,0.678249,1.591464,-1.419193,-0.714796
453,927241,1,1.838967,2.274081,1.979530,1.753731,1.479944,3.302377,3.342871,2.614019,...,1.962444,2.196646,2.294064,1.665612,1.413392,3.916770,3.209630,2.241080,2.036887,2.295030


In [46]:
a = data_train_df.head(10)
a.to_csv('prueba.csv')

###### Apply smote

In [47]:
smote = SMOTE(random_state=42)
train_data_upsampled, diagnosis_res = smote.fit_resample(data_train_df, diagnosis)
train_data_upsampled["diagnosis"].value_counts()

1    287
0    287
Name: diagnosis, dtype: int64

In [48]:
train_y = ((data_train_df.iloc[:, 1] == 0) + 0).to_numpy()
train_X = data_train_df.iloc[:, 2:].to_numpy()

val_y = ((data_val_df.iloc[:, 1] == 0) + 0).to_numpy()
val_X = data_val_df.iloc[:, 2:].to_numpy()

test_y = ((data_test_df.iloc[:, 1] == 0) + 0).to_numpy()
test_X = data_test_df.iloc[:, 2:].to_numpy();

In [49]:
train_y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1,

###### SageMaker converts each observation in the dataset into a binary representation, and then upload this data to S3

###### I start with the train set

In [50]:
train_file = "linear_train.data"
#Manipulate bytes data in memory
f = io.BytesIO()
#I want to upload the model in Sagemaker so I can perform next operation like
smac.write_numpy_to_dense_tensor(f, train_X.astype("float32"), train_y.astype("float32"))
#the function seek()  is like a cursor, which defines from where the data has to be read or written in the file. 
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "train", train_file)
).upload_fileobj(f)

###### then with the validation set

In [51]:
validation_file = "linear_validation.data"

f = io.BytesIO()
smac.write_numpy_to_dense_tensor(f, val_X.astype("float32"), val_y.astype("float32"))
f.seek(0)

boto3.Session().resource("s3").Bucket(bucket).Object(
    os.path.join(prefix, "validation", validation_file)
).upload_fileobj(f)

# 3. Train

###### Specify container images used for training and hosting SageMaker's linear-learner

In [52]:
from sagemaker import image_uris

container = image_uris.retrieve(region=boto3.Session().region_name, framework="linear-learner")

In [53]:
linear_job = "linear-" + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

print("Job name is:", linear_job)

linear_training_params = {
    "RoleArn": role,
    "TrainingJobName": linear_job,
    "AlgorithmSpecification": {"TrainingImage": container, "TrainingInputMode": "File"},
    "ResourceConfig": {"InstanceCount": 1, "InstanceType": "ml.c4.2xlarge", "VolumeSizeInGB": 10},
    "InputDataConfig": [
        {
            "ChannelName": "train",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/train/".format(bucket, prefix),
                    "S3DataDistributionType": "ShardedByS3Key",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
        {
            "ChannelName": "validation",
            "DataSource": {
                "S3DataSource": {
                    "S3DataType": "S3Prefix",
                    "S3Uri": "s3://{}/{}/validation/".format(bucket, prefix),
                    "S3DataDistributionType": "FullyReplicated",
                }
            },
            "CompressionType": "None",
            "RecordWrapperType": "None",
        },
    ],
    "OutputDataConfig": {"S3OutputPath": "s3://{}/{}/".format(bucket, prefix)},
    "HyperParameters": {
        "feature_dim": "30",
        "mini_batch_size": "100",
        "predictor_type": "regressor",
        "epochs": "10",
        "num_models": "32",
        "loss": "absolute_loss",
    },
    "StoppingCondition": {"MaxRuntimeInSeconds": 60 * 60},
}

Job name is: linear-2022-10-19-04-31-13


In [54]:
%%time

region = boto3.Session().region_name
sm = boto3.client("sagemaker")

sm.create_training_job(**linear_training_params)

status = sm.describe_training_job(TrainingJobName=linear_job)["TrainingJobStatus"]
print(status)
sm.get_waiter("training_job_completed_or_stopped").wait(TrainingJobName=linear_job)
if status == "Failed":
    message = sm.describe_training_job(TrainingJobName=linear_job)["FailureReason"]
    print("Training failed with the following error: {}".format(message))
    raise Exception("Training job failed")

InProgress
CPU times: user 90 ms, sys: 7.79 ms, total: 97.8 ms
Wall time: 6min


# 4. Host

##### Now that we've trained the linear algorithm on our data, let's setup a model which can later be hosted.

In [55]:
linear_hosting_container = {
    "Image": container,
    "ModelDataUrl": sm.describe_training_job(TrainingJobName=linear_job)["ModelArtifacts"][
        "S3ModelArtifacts"
    ],
}

create_model_response = sm.create_model(
    ModelName=linear_job, ExecutionRoleArn=role, PrimaryContainer=linear_hosting_container
)

print(create_model_response["ModelArn"])

arn:aws:sagemaker:us-east-1:279310279785:model/linear-2022-10-19-04-31-13


###### configure endpoints

In [56]:
linear_endpoint_config = "DEMO-linear-endpoint-config-" + time.strftime(
    "%Y-%m-%d-%H-%M-%S", time.gmtime()
)
print(linear_endpoint_config)
create_endpoint_config_response = sm.create_endpoint_config(
    EndpointConfigName=linear_endpoint_config,
    ProductionVariants=[
        {
            "InstanceType": "ml.m4.xlarge",
            "InitialInstanceCount": 1,
            "ModelName": linear_job,
            "VariantName": "AllTraffic",
        }
    ],
)

print("Endpoint Config Arn: " + create_endpoint_config_response["EndpointConfigArn"])

DEMO-linear-endpoint-config-2022-10-19-04-37-50
Endpoint Config Arn: arn:aws:sagemaker:us-east-1:279310279785:endpoint-config/demo-linear-endpoint-config-2022-10-19-04-37-50


###### Create endpoints

In [57]:
%%time

linear_endpoint = "DEMO-linear-endpoint-" + time.strftime("%Y%m%d%H%M", time.gmtime())
print(linear_endpoint)
create_endpoint_response = sm.create_endpoint(
    EndpointName=linear_endpoint, EndpointConfigName=linear_endpoint_config
)
print(create_endpoint_response["EndpointArn"])

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Status: " + status)

sm.get_waiter("endpoint_in_service").wait(EndpointName=linear_endpoint)

resp = sm.describe_endpoint(EndpointName=linear_endpoint)
status = resp["EndpointStatus"]
print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

if status != "InService":
    raise Exception("Endpoint creation did not succeed")

DEMO-linear-endpoint-202210190437
arn:aws:sagemaker:us-east-1:279310279785:endpoint/demo-linear-endpoint-202210190437
Status: Creating
Arn: arn:aws:sagemaker:us-east-1:279310279785:endpoint/demo-linear-endpoint-202210190437
Status: InService
CPU times: user 126 ms, sys: 10.1 ms, total: 136 ms
Wall time: 4min 31s


# 5. Predict

In [58]:
#Predict on Test Data
#Now that we have our hosted endpoint, we can generate statistical predictions from it.
#Function to convert an array to a csv
def np2csv(arr):
    csv = io.BytesIO()
    np.savetxt(csv, arr, delimiter=",", fmt="%g")
    return csv.getvalue().decode().rstrip()

In [59]:
#we'll invoke the endpoint to get predictions.
runtime = boto3.client("runtime.sagemaker")

payload = np2csv(test_X)
response = runtime.invoke_endpoint(
    EndpointName=linear_endpoint, ContentType="text/csv", Body=payload
)
result = json.loads(response["Body"].read().decode())
test_pred = np.array([r["score"] for r in result["predictions"]])


In [60]:
test_pred

array([ 0.04226309,  1.0365479 ,  0.1507661 ,  0.90998429,  0.86867291,
        0.49754533,  1.11673665,  0.85289586,  0.80492091,  0.97770917,
        0.76906997,  0.49509937,  0.92538774, -0.23574013,  0.5695489 ,
        0.45155555,  0.82817793,  0.27650058,  0.52302068,  0.74418294,
        0.97209132, -0.3204208 ,  0.05093843,  0.71611863, -0.19526899,
        0.41327184,  0.83698159,  1.16516709,  0.80906522,  0.95762062,
        0.92193389,  1.07709646,  0.95696694,  0.95684934, -0.30172652,
        0.97497308,  0.13155583,  0.63731271,  0.9815312 ,  1.01691246,
        0.88042367,  0.91124701,  1.1535306 ,  0.70132971,  0.81002271,
        0.20253134,  0.96383226,  0.53919709,  0.68055457,  0.6362685 ,
        0.16792595, -0.19941622,  0.48623624,  0.63510758,  0.25722539,
        0.03867632,  1.00513995,  0.91763091])

In [49]:
#compare linear learner based mean absolute prediction errors from a baseline prediction which uses majority class to predict every instance.
test_mae_linear = np.mean(np.abs(test_y - test_pred))
test_mae_baseline = np.mean(
    np.abs(test_y - np.median(train_y))
)  ## training median as baseline predictor

print("Test MAE Baseline :", round(test_mae_baseline, 3))
print("Test MAE Linear:", round(test_mae_linear, 3))

Test MAE Baseline : 0.429
Test MAE Linear: 0.212


In [50]:
#Let's compare predictive accuracy using a classification threshold of 0.5 for the predicted and compare against 
#the majority class prediction from training data set.
test_pred_class = (test_pred > 0.5) + 0
test_pred_baseline = np.repeat(np.median(train_y), len(test_y))

prediction_accuracy = np.mean((test_y == test_pred_class)) * 100
baseline_accuracy = np.mean((test_y == test_pred_baseline)) * 100

print("Prediction Accuracy:", round(prediction_accuracy, 1), "%")
print("Baseline Accuracy:", round(baseline_accuracy, 1), "%")

Prediction Accuracy: 85.7 %
Baseline Accuracy: 57.1 %
